### **6.12 - How to react in case of failure_ or retry**

Es muy probable que si tu tarea falla, quieras hacer algo. Tal vez sólo quieras reintentar tu tarea o tal vez quieras hacer algo más complejo. Bueno, en ese caso hay dos maneras de hacer esto, y la primera es "aprovechar las trigger rules". Así, por ejemplo, podrías tener algunas tareas que se dispararán sólo si la tarea upstream, por lo que la tarea en cuestión ha fallado, o podrías aprovechar las "callbacks", y eso es lo que vas a descubrir ahora. ¿Qué es la "callback"? Bueno, "callback" no es más que una función, que será llamada de acuerdo a un evento. Por ejemplo, si tu tarea falla, entonces hay un "callback" que será llamado. O si tu tarea es reintentada de nuevo, hay un "callback" para eso. O si tu DagRun no tuvo exito, usaras un "callback". Por lo tanto, hay dos niveles en los que se pueden definir "callbacks" en Airflow, y el primer nivel es en el nivel DAG. Por ejemplo, usted quiere hacer algo si su DagRun tiene éxito, entonces en ese caso puede utilizar "on_success_callback", que espera una función de Python, por ejemplo, "success_callback". 

<center><img src="https://i.postimg.cc/3xTrWqcF/a1467.png"></center>

Obviamente, necesitas crear la función correspondiente, así que escribe "def _success_callback( )" y esta función tiene un único argumento que es "context", y context aquí es exactamente como con el PythonOperator, contiene mucha información sobre tu DAG, así que, por ejemplo, si quieres acceder a la "execution_date" de tu DAG, o si quieres acceder a los parámetros de tu DAG, puedes usar "context". Aquí, puedes simplemente imprimir "context" y si te preguntas dónde podrás ver la salida de ese comando, tendrás que echar un vistazo a los "logs" de tu DAG en la carpeta "logs". 

<center><img src="https://i.postimg.cc/zGFzhjmZ/a1468.png"></center>

Ahora, algo más interesante que ser advertido, si tu DagRun tiene éxito, ¿qué pasa si tu DagRun no tuvo éxito? ¿Qué pasa si tu DagRun falla? Bueno, en ese caso, puedes usar el callback "on_failure_callback", que espera una función python "_failure_callback", y obviamente necesitas crear la función. Así que vamos a hacer esto "def _failure_callback(context)" y puedes imprimir el context.

<center><img src="https://i.postimg.cc/MGZWmQxc/a1469.png"></center>

Bueno, esto es sólo un ejemplo porque deberías hacer algo mucho más significativo que simplemente imprimir el context en la salida estándar. Por ejemplo, enviar un correo electrónico o enviar una notificación de Slack. Sólo ten en cuenta, una cosa, si por alguna razón, tienes un fallo en tu callback, dentro de tu callback, entonces tu callback no se reintentará automáticamente, ¿de acuerdo? Eso es lo único que tienes que recordar. Así que trata de hacer algo muy simple en tu callback. Ahora, esto era para el nivel DAG, pero puedes definir callbacks a nivel de tarea. Veamos cómo hacerlo. Para tus tareas, puedes usar tres callbacks diferentes, el primero es "on_success_callback", el segundo "on_failure_callback" y el último es "on_retry_callback". Déjame mostrarte cómo usarlos. Así que, digamos, que deseas ser advertido de los tres eventos. Entonces, primero necesitas crear los callbacks correspondientes. Vamos a escribir "def _" y este, queremos ser advertidos de las "extract tasks". Así que vamos a escribir, "def _extract_callback_success", y de nuevo, éste sólo toma un único argumento, que es "context". Entonces podemos imprimir "SUCCESS CALLBACK". A continuación, crear dos funciones más, una para el fracaso y la última para reintentar.

<center><img src="https://i.postimg.cc/wBW9fjTw/a1470.png"></center>

Así que, obviamente, quieres ser avisado si la tarea "extract" ha tenido éxito, entonces defines "on_success_callback" igual a "_extract_callback_success". Si la tarea ha fallado, entonces debes usar el argumento "on_failure_callback" igual a "_extract_callback_failure". Luego el último, que es "on_retry_callback". Puede ser interesante para ti si quieres hacer algo entre cada reintento, cada vez que tu tarea es reintentada, si quieres hacer algo, entonces puedes usar ese callback. Por lo tanto, aquí se especifica "_extract_callback_retry". Es tan simple como eso. 

<center><img src="https://i.postimg.cc/SsJkwbnk/a1471.png"></center>
<center><img src="https://i.postimg.cc/3RVYhfGp/a1472.png"></center>

Tienes tres callbacks diferentes para cada una de tus tareas. "on success callback", "on failure callback" y "on retry callback". Ahora, hay dos cosas de las que me gustaría hablar, y la primera es ¿qué pasa si quieres identificar si tu tarea no tuvo éxito en el "on_failure_callback" debido a un "timeout"? Bueno, puedes hacerlo y déjame mostrarte cómo. Si vuelves a tu callback "extract_callback_failure" justo aquí

<center><img src="https://i.postimg.cc/yxRB8hhG/a1473.png"></center>

puedes echar un vistazo a tu diccionario "context" y si tu tarea no tuvo éxito debido a un "timeout" o en realidad debido a un error, entonces recibirás una "exception". Así que el primer paso es echar un vistazo a tu diccionario "context", si la key "exception" existe:

<center><img src="https://i.postimg.cc/RCJvd9rT/a1474.png"></center>

Si es así, entonces puedes filtrar en esa excepción. Por ejemplo, si "context["exception"]" es una instancia de AirflowTaskTimeout

<center><img src="https://i.postimg.cc/PrGTFcTn/a1475.png"></center>

Y si es así, significa que su tarea no tuvo éxito debido a un timeout. O puedes filtrar la misma "exception", pero esta vez, si es un sensor. De nuevo, tienes "AirflowSensorTimeout" y así sucesivamente.

<center><img src="https://i.postimg.cc/FsjNtZHd/a1476.png"></center>

Tienes un montón de excepciones y esto es realmente útil para ti ya que puedes actuar de acuerdo a la excepción que obtuviste. Y para acceder a esas excepciones, necesitas importarlas:

<center><img src="https://i.postimg.cc/fRCsyRXM/a1477.png"></center>

Ahora, la segunda cosa de la que me gustaría hablar es, digamos que quieres saber el número de reintentos que ha habido, digamos que quieres saber el número de veces que tu tarea ha sido reintentada en tu callback "retry". Bueno, puedes hacer eso. Lo único que tienes que hacer es acceder al objeto instancia de la tarea (task instance object) y más concretamente a la propiedad "try_numbers". Así que aquí se podría decir:

<center><img src="https://i.postimg.cc/8zNDT67y/a1478.png"></center>

Así que los callbacks son extremadamente útiles. Obviamente, te ayudan a reaccionar de acuerdo a un evento si tu tarea falla, si tu DagRun no tuvo éxito, si tu tarea es reintentada y así sucesivamente y puedes acceder a alguna información adicional, por ejemplo, el número de veces que tu tarea ha sido reintentada o la excepción lanzada por tu tarea cuando no tuvo éxito.